# Karp-Miller-Rosenberg Algorithm (KMR)

## Constructing DBF

DBF consists of name and pos (sometimes referred as first_entry) tables

Note that I start indexing from 0 (conventionally)

In [1]:
import kmr

seq = [(1, 2), (3, 1), (2, 2), (1, 1), (2, 3), (1, 2)]

kmr.sort_rename(seq)

([1, 4, 2, 0, 3, 1], {0: 3, 1: 0, 2: 2, 3: 4, 4: 1})

In [2]:
text = "abbabbaba"
dbf = kmr.kmr(text)

kmr.show_dbf(dbf, text)


names
1 [0, 1, 1, 0, 1, 1, 0, 1, 0]
2 [0, 3, 2, 0, 3, 2, 0, 2, 1]
4 [1, 6, 4, 1, 6, 3, 0, 5, 2]

pos
1 [1, 2]
2 [1, 9, 3, 2]
4 [7, 1, 9, 6, 3, 8, 2]


In [3]:
text = "abbabbaba"
dbf = kmr.kmr(text)

kmr.show_dbf(dbf, text)


names
1 [0, 1, 1, 0, 1, 1, 0, 1, 0]
2 [0, 3, 2, 0, 3, 2, 0, 2, 1]
4 [1, 6, 4, 1, 6, 3, 0, 5, 2]

pos
1 [1, 2]
2 [1, 9, 3, 2]
4 [7, 1, 9, 6, 3, 8, 2]


In [4]:
text = "abaabbaa"
dbf = kmr.kmr(text)
kmr.show_dbf(dbf, text)


names
1 [0, 1, 0, 0, 1, 1, 0, 0]
2 [1, 3, 0, 1, 4, 3, 0, 2]
4 [2, 5, 0, 3, 7, 6, 1, 4]

pos
1 [1, 2]
2 [3, 1, 8, 2, 5]
4 [3, 7, 1, 4, 8, 2, 6, 5]


Looking good..

## Searching 

I have included two implementations:
- basic - constructs DBF(pattern&text) for each search which yields $O(n \cdot log(m))$ space and time complexity ($log(m)$ since it only needs part of DBF)
- efficient - uses constructed DBF and by using binary search finds the name for pattern in O($m \cdot log(n))$ and having that name it traverses name table searching for other occurencs. This implementations yields $O(n\cdot log(n))$ space complexity and time complexity for constructing DBF, but $O(m \cdot log(n) + n)$ for every following search.

 Note that while I was construcing pos table I could store all positions instead of just the first one, which would slighlty increase space usage (we'd still stay in $O(nlogn)$) but essentially it's a tradeoff between space and time. I chose (slighlty) bigger runtime using (slightly) less space.

Well searching uses basic facts mentioned at the lecture so there's no point in bringing it up again. 
- n = len(text)
- m = len(pattern)

Below I have shown few examples..

In [5]:
text = 'aaaa'
pattern = 'a'

print(list(kmr.basic_search(pattern, text)))

dbf = kmr.kmr(text)
print(list(kmr.efficient_search(pattern, text, dbf)))

[0, 1, 2, 3]
[0, 1, 2, 3]


In [6]:
text = "abaabbaa"
pattern = 'ab'

print(list(kmr.basic_search(pattern, text)))

dbf = kmr.kmr(text)
print(list(kmr.efficient_search(pattern, text, dbf)))

[0, 3]
[0, 3]


In [7]:
## Example for pattern whose length is not power of 2. Note that we use previously construced dbf for efficient search
pattern = 'bba'

print(list(kmr.basic_search(pattern, text)))
print(list(kmr.efficient_search(pattern, text, dbf)))

[4]
[4]


In [8]:
text = "ahdshsajkfdabhabfdasfarfqewa"
pattern = 'fqewa'

print(list(kmr.basic_search(pattern, text)))

dbf = kmr.kmr(text)
print(list(kmr.efficient_search(pattern, text, dbf)))

[23]
[23]


Looking good..

## Time benchmarks for constructing DBF

In [9]:
def read_text(filename):
    with open('resources' + '/' + filename, encoding='utf-8') as file:
        return file.read()

text1 = read_text('1997_714.txt')
text2 = read_text('romeo-i-julia-700.txt')
text3  = read_text('zad6')

texts = [text1, text2, text3]
names = ['1997', 'romeo', 'zad6']

In [10]:
from suffix_tree import SuffixTree

for i in range(2, -1, -1):
    print(f'Testing {names[i]}', end='\n\n')
    print('DBF: ')
    %timeit kmr.kmr(texts[i])
    print('Suffix Tree: ')
    %timeit SuffixTree(texts[i])
    print('-' * 48)

Testing zad6

DBF: 
33.6 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Suffix Tree: 
7.29 ms ± 339 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
------------------------------------------------
Testing romeo

DBF: 
850 ms ± 46.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Suffix Tree: 
113 ms ± 4.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
------------------------------------------------
Testing 1997

DBF: 
24.8 s ± 1.71 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
Suffix Tree: 
2.79 s ± 201 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
------------------------------------------------


We can see the difference in running time: O(n) for suffix trees and $O(n \cdot log(n))$ for DBF

## Space benchmarks

I guess the easiest way to do it was to serialize object to file and measure the size of file - so that's what I did. Of course that way it'll not be exactly the same size as python object's but we'll still be able to see order of magnitude of size.

In [12]:
import pickle
import os

def dump(serializable, name):
    path = 'resources/' + name + '.pickle'
    with open(path, "wb") as file:
        pickle.dump(serializable, file)

    return path

def filesize_kb(path):
    return os.stat(path).st_size / 1024

In [21]:
textfile_size = []
for filename in ['1997_714.txt', 'romeo-i-julia-700.txt', 'zad6']:
    path = 'resources' + '/' + filename
    textfile_size.append(filesize_kb(path))

for i in range(2, -1, -1):
    print(f'Testing {names[i]}', end='\n\n')
    dbf = kmr.kmr(texts[i])
    
    dbf_size = filesize_kb(dump(dbf, names[i]))
    text_size = textfile_size[i]

    print(f'DBF size:      {dbf_size} kB')
    print(f'Textfile size: {text_size} kB')
    print(f'Ratio:         {dbf_size / text_size}')

    print('-' * 48)

Testing zad6

DBF size:      66.1142578125 kB
Textfile size: 0.9248046875 kB
Ratio:         71.48996832101373
------------------------------------------------
Testing romeo

DBF size:      1512.265625 kB
Textfile size: 13.875 kB
Ratio:         108.99211711711712
------------------------------------------------
Testing 1997

DBF size:      52697.1455078125 kB
Textfile size: 248.1767578125 kB
Ratio:         212.33715023235865
------------------------------------------------


We can see that even without storing all positions of name, we consume a lot of space, so this is probably not a good idea to use this algorithm on vast textfiles.

## Time comparison with KMP

In [22]:
db1 = kmr.kmr(text1)
db2 = kmr.kmr(text2)
db3 = kmr.kmr(text3)

In [60]:
from kmp import KMPSearch

def benchmark(text, pattern, dbf, name, show_result = True):
    print(f'Testing {name}', end='\n\n')
    
    if show_result:
        print('KMR result: ', end = '')
        print(list(kmr.efficient_search(pattern, text, dbf)))

        print('KMP result: ', end = '')
        print(list(KMPSearch(pattern, text)))
    else:
        print('KMR result: ', end = '')
        print(len(list(kmr.efficient_search(pattern, text, dbf))), 'matches')

        print('KMP result: ', end = '')
        print(len(list(KMPSearch(pattern, text))), 'matches')

    print('KMR time:')
    %timeit list(kmr.efficient_search(pattern, text, dbf))

    print('KMP time:')
    %timeit list(KMPSearch(pattern, text))

    print('-' * 64)


In [61]:
text = text2
dbf = db2
pattern = 'a'

benchmark(text, pattern, dbf, names[1], False)

Testing romeo

KMR result: 644 matches
KMP result: 644 matches
KMR time:
1.09 ms ± 5.97 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
KMP time:
3.36 ms ± 42.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
----------------------------------------------------------------


In [50]:
text = text3
dbf = db3
pattern = 'DBF'

benchmark(text, pattern, dbf, names[2])

Testing zad6

KMR result: [113, 245, 300, 401, 484]
KMP result: [113, 245, 300, 401, 484]
KMR time:
211 µs ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
KMP time:
252 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
----------------------------------------------------------------


In [52]:
text = text2
dbf = db2
pattern = 'PARYS'

benchmark(text, pattern, dbf, names[1])

Testing romeo

KMR result: [134, 627]
KMP result: [134, 627]
KMR time:
3.52 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
KMP time:
4.08 ms ± 442 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
----------------------------------------------------------------


In [82]:
text = text1
dbf = db1
pattern = 'Osoby fizyczne'

benchmark(text, pattern, dbf, names[0], False)


Testing 1997

KMR result: 1 matches
KMP result: 1 matches
KMR time:
75.3 ms ± 2.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
KMP time:
65.8 ms ± 630 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
----------------------------------------------------------------


In [83]:
text = text1
dbf = db1
pattern = 'podatku'

benchmark(text, pattern, dbf, names[0], False)


Testing 1997

KMR result: 67 matches
KMP result: 67 matches
KMR time:
74.1 ms ± 659 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
KMP time:
64.6 ms ± 805 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
----------------------------------------------------------------


In [75]:
pattern = 'Ryczałt od przychodów ewidencjonowanych opłacają również podatnicy, którzy\n  w roku poprzedzającym rok podatkowy prowadzili działalność samodzielnie lub\n  w formie spółki, z której przychody były opodatkowane wyłącznie w formie\n  karty podatkowej, lub za część roku były opodatkowane w formie karty\n  podatkowej i za część roku na ogólnych zasadach, a łączne przychody w roku\n  poprzedzającym rok podatkowy nie przekroczyły kwoty 400.000 zł;'

text = text1
dbf = db1

benchmark(text, pattern, dbf, names[0])

Testing 1997

KMR result: [9728]
KMP result: [9728]
KMR time:
130 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
KMP time:
86.7 ms ± 13.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
----------------------------------------------------------------


In [77]:
text = text1
dbf = db1
pattern = 'ala'

benchmark(text, pattern, dbf, names[0], False)


Testing 1997

KMR result: 62 matches
KMP result: 62 matches
KMR time:
84.6 ms ± 6.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
KMP time:
88 ms ± 7.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
----------------------------------------------------------------


### Conclusion

Well for one thing it looks like our implementation is correct. For short patterns KMR turned out to be faster than KMP, for others special, longer ones (like this long sentence above) KMR turned out to be slower. Well this might be becasuse I didn't store all positions (which as we can see wasn't needed to beat KMP for smaller patterns). If we look at asymptotic time complexity we can see why this is the case: 

$$
    O(n + m) \text{ vs } O(n + m \cdot log(n))
$$

If we were to remember all positions (in pos table) then we'd have

$$
    O(m \cdot log(n))
$$

But as I said previously this is just a tradeoff between space and time.

Overall bcs of huge space complexity and long preprocessing (DBF constructing) time it does not make sense to use KMR for pattern matching, but variations to this algorithm have other applications.